In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numba
from py_vollib import black_scholes_merton
from py_vollib.black_scholes_merton.implied_volatility import implied_volatility

import plotly.graph_objects as go
import plotly.express as px
from scipy.interpolate import griddata

In [10]:
s = 1.0    # spot price S_0
r = 0.0    # zero interest rate

In [46]:
### YOUR CODE HERE
# Read the data, by using pandas
df = pd.read_csv('data.txt', sep=",")
df = df.rename(columns={'expiration': 'T', ' strike': 'K', ' call_price': 'C'})

# The local volatility surface can be computed from a call option price surface that can be 
# constructed by market quotes of call option prices 

q = 0 # interest rate of divedents

#define the array of volatilities sigma_iv and compute them
sigma_iv = [[0] for i in range(df['K'].shape[0])]
for i in range(len(sigma_iv)):
    try:
        sigma_iv[i] = implied_volatility(df["C"][i], 1, df["K"][i], df["T"][i], r, q, 'c')
    except:
        sigma_iv[i] = 0
#add him to our Dataset df
df["sigma_iv"] = sigma_iv
# df = df.astype(float)

In [47]:
#plot the 3d scatter of dots, which make a surface of volatility
fig = px.scatter_3d(df, x='K', y='T', z='sigma_iv')
fig.show()

In [48]:
#define x,y,z for next plot
x = np.array(df["K"])
y = np.array(df["T"])
z = np.array(df["sigma_iv"])

#smoothing
xi = np.linspace(x.min(), x.max(), 100)
yi = np.linspace(y.min(), y.max(), 100)
X,Y = np.meshgrid(xi,yi)
Z = griddata((x,y),z,(X,Y), method='cubic')

#plot the surface of volatility
fig = go.Figure(go.Surface(x=xi,y=yi,z=Z))
fig.show()


In [49]:
### Use "lets be rational" implementation of implied volatility calculation from https://github.com/vollib/py_vollib

### YOUR CODE HERE
K_uniq = list(dict.fromkeys(df['K'].values))
T_uniq = list(dict.fromkeys(df['T'].values))

k_len = len(K_uniq)
t_len = len(T_uniq)

table = np.array([[sigma_iv[i+j*k_len] for i in range(k_len)] for j in range(t_len)])

# F_T = np.array([s * np.exp(r*T_uniq[i]) for i in range(t_len)])
F_T = s
w = np.array([[table[j][i]**2 * T_uniq[j] for i in range(k_len)] for j in range(t_len)])
y = np.array([np.log(K_uniq[j]/F_T) for j in range(k_len)])

dw_dy = np.zeros((t_len, k_len))
d2w_dy2 = np.zeros((t_len, k_len))
sigma_dup = np.zeros((t_len, k_len))

for i in range(t_len):
    for j in range(k_len):
        # As I thought, we need to make a dervatives
        if j == 0:
            dw_dy[i, j] = (w[i, j + 1] - w[i, j]) / (y[j + 1] - y[j])
        elif j == len(y) - 1:
            dw_dy[i, j] = (w[i, j] - w[i, j - 1]) / (y[j] - y[j - 1])
        else:
            dw_dy[i, j] = (w[i, j + 1] - w[i, j - 1]) / (y[j + 1] - y[j - 1])

for i in range(t_len):
    for j in range(k_len):
        if j == 0:
            d2w_dy2[i, j] = (dw_dy[i, j + 1] - dw_dy[i, j]) / (y[j + 1] - y[j])
        elif j == len(y) - 1:
            d2w_dy2[i, j] = (dw_dy[i, j] - dw_dy[i, j - 1]) / (y[j] - y[j - 1])
        else:
            d2w_dy2[i, j] = (dw_dy[i, j + 1] - dw_dy[i, j - 1]) / (y[j + 1] - y[j - 1])

        if i == 0:
            sigma_dup[i, j] = ((w[i + 1, j] - w[i, j]) / (T_uniq[i + 1] - T_uniq[i])) / (
                        1 - dw_dy[i, j] * y[j] / w[i, j] + d2w_dy2[i, j] / 2 + dw_dy[i, j] ** 2 / 4 * (
                            -1 / 4 + 1 / w[i, j] + (y[j] / w[i, j]) ** 2))
        elif i == len(T_uniq) - 1:
            sigma_dup[i, j] = ((w[i, j] - w[i - 1, j]) / (T_uniq[i] - T_uniq[i - 1])) / (
                        1 - dw_dy[i, j] * y[j] / w[i, j] + d2w_dy2[i, j] / 2 + dw_dy[i, j] ** 2 / 4 * (
                            -1 / 4 + 1 / w[i, j] + (y[j] / w[i, j]) ** 2))
        else:
            sigma_dup[i, j] = ((w[i + 1, j] - w[i - 1, j]) / (T_uniq[i + 1] - T_uniq[i - 1])) / (
                        1 - dw_dy[i, j] * y[j] / w[i, j] + d2w_dy2[i, j] / 2 + dw_dy[i, j] ** 2 / 4 * (
                            -1 / 4 + 1 / w[i, j] + (y[j] / w[i, j]) ** 2))

df["sigma_dup"] = np.ravel(sigma_dup)

In [50]:
x = np.array(df["K"])
y = np.array(df["T"])
z = np.array(df["sigma_dup"])

xi = np.linspace(x.min(), x.max(), 100)
yi = np.linspace(y.min(), y.max(), 100)
X,Y = np.meshgrid(xi,yi)
Z = griddata((x,y),z,(X,Y), method='cubic')

fig = go.Figure(go.Surface(x=xi,y=yi,z=Z))
fig.show()


In [51]:
fig = px.scatter_3d(df, x='K', y='T', z='sigma_dup')
fig.show()